In [ ]:
import numpy as np
from numpy import array as arr
from numpy import zeros as zarr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.runSettings import load_settings
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import ROOT

In [ ]:
data = {}
runs = arr([184,190,192])
thresholds = {
    184:arr([40,80,100,140,180,220,260,300,340,400,450,500,600]),
    190:arr([40,80,100,140,180,220,260,300,340,400,450,500,600]),
    192:arr([50,100,150,200,250,300,350,400,430,460,480,510,540,570,600])
}
for run in runs:
    data[run] = {
        "thr":thresholds[run],
        "eff":zarr([len(thresholds[run]),3]), # [eff, uncDown, uncUp]
        "fakeRate":zarr([len(thresholds[run]),2]), # [fakeRate, unc]
        "settings":load_settings(run)
    }
    
FilePath = "/home/jona/DESY/analysis_TB/output/"


for run in runs:
    for i_thr, thr in enumerate(data[run]["thr"]):
        FileName = FilePath + str(run) + "_thr-" + str(thr) + ".root"
        rFile = ROOT.TFile(FileName)
        if not rFile:
            print("File not found")
            rFile.Close()
            continue
        
        # Get the efficiency
        rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0")
        if not rDir:
            print("Dir not found")
        else:
            TEff = rDir.Get("eTotalEfficiency")    
            if not TEff:
                print("Efficiency not found")
            else:
                data[run]["eff"][i_thr,0] = TEff.GetEfficiency(1)
                data[run]["eff"][i_thr,1] = TEff.GetEfficiencyErrorLow(1)
                data[run]["eff"][i_thr,2] = TEff.GetEfficiencyErrorUp(1)
        
        # Get the fake rate
        rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0/fake_rate")
        if not rDir:
            print("fake_rate: Dir not found")
        else:
            hFakePixelPerEvt = rDir.Get("hFakePixelPerEvent")
            if not hFakePixelPerEvt:
                print("fake_rate: hFakePixelPerEvent not found")
            else:
                data[run]["fakeRate"][i_thr,0] = hFakePixelPerEvt.GetMean()
                data[run]["fakeRate"][i_thr,1] = hFakePixelPerEvt.GetRMS()
    
        rFile.Close() 

In [ ]:
for run in runs:
    
    fig, ax = Hist.create_fig(1,1)
    
    x = data[run]["thr"]
    y = data[run]["eff"][:,0]
    yerr = arr([data[run]["eff"][:,1],data[run]["eff"][:,2]])
    ax.errorbar(x, y, yerr=yerr, fmt='.-', color="black", lw=0.8, ms=2)
    ax.axhline(0.99, color="black", lw=0.5, ls="--", alpha=0.5)
    
    # ax.set_xlabel(r"Threshold / $e^-$", loc="right")
    # ax.set_ylabel("Efficiency", loc="top")
    ax.set_xlim(30,300)
    ax.set_ylim(0.9,1.005)
    
    ax2 = ax.twinx()
    color = 'darkred'
    ax2.spines['right'].set_color(color)
    ax2.set_ylabel('Fake rate / pix per evt', color=color, loc="top")
    ax2.tick_params(axis='y', labelcolor=color, direction="in", color=color)
    
    y = data[run]["fakeRate"][:,0]
    yerr = data[run]["fakeRate"][:,1]
    ax2.errorbar(x, y, yerr=yerr, fmt='.-', color=color, lw=.8, ms=2)
    
    names = {
        184:"medium", 190:"low", 192:"high"
    }
    Hist.finalize(data[run]["settings"], fig, ax, title="Efficiency\n("+names[run]+ " Krumm. current)", xlabel=r"Threshold / $e^-$", ylabel="Efficiency", param_narrow=True, param_fontsize=8, grid=False, subplots_adjust=[0.,0.9,0.88], tight_layout=False)